In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
from glob import glob

### 1단계: 파일 시스템 모듈

In [1]:
# import black
# # 설치 확인
# !pip list | grep black
# !pip list | grep autopep8
# # Black 포맷터 로드
# %load_ext nb_black

# # 또는 autopep8 사용하려면
# %load_ext autopep8

In [27]:
import os
from typing import List, Tuple, Optional
from pathlib import Path

# 주피터 노트북 출력 제한 해제 설정
import sys

# IPython 설정 (주피터 노트북에서만 동작)
try:
    from IPython.core.getipython import get_ipython
    ipython = get_ipython()
    if ipython:
        ipython.ast_node_interactivity = "all"
except:
    pass

# pandas 출력 제한 해제 (있는 경우)
try:
    import pandas as pd
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
except ImportError:
    pass

print("출력 제한 해제 설정 완료")

출력 제한 해제 설정 완료


In [28]:
def scan_directory(root_path: str) -> List[str]:
    """
    디렉토리를 스캔하여 .md 파일 경로를 찾습니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[str]: 발견된 .md 파일의 절대 경로 리스트
    
    Example:
        >>> scan_directory("./manual/user/firstUser")
        ['/path/to/manual/user/firstUser/login/login.md', 
         '/path/to/manual/user/firstUser/approval/approval.md']
    """
    md_files = []
    
    if not os.path.exists(root_path):
        print(f"⚠️ 경로가 존재하지 않습니다: {root_path}")
        return md_files
    
    for root, dirs, files in os.walk(root_path):
        #print(root,dirs,files)
        
        for file in files:
            if file.endswith('.md'):
                file_path = os.path.join(root, file)
                md_files.append(os.path.abspath(file_path))
    
    return sorted(md_files)

In [29]:
def read_markdown_file(file_path: str) -> Optional[str]:
    """
    마크다운 파일의 내용을 읽습니다.
    
    Args:
        file_path (str): 읽을 마크다운 파일 경로
    
    Returns:
        Optional[str]: 파일 내용 (실패 시 None)
    
    Example:
        >>> content = read_markdown_file("./manual/user/firstUser/login/login.md")
        >>> print(content[:50])
        # Login
        
        ---
        ## 목차
        1. [로그인 페이지](#1-로그인-페이지)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
        return None
    except UnicodeDecodeError:
        print(f"❌ 파일 인코딩 오류: {file_path}")
        return None
    except Exception as e:
        print(f"❌ 파일 읽기 오류: {file_path} - {str(e)}")
        return None

In [30]:
def extract_path_info(file_path: str, root_path: str) -> Tuple[str, str, str]:
    """
    파일 경로에서 경로 정보를 추출합니다.
    
    Args:
        file_path (str): 전체 파일 경로
        root_path (str): 루트 디렉토리 경로
    
    Returns:
        Tuple[str, str, str]: (상대경로, 파일명_확장자제거, 디렉토리명)
    
    Example:
        >>> extract_path_info("/path/manual/user/firstUser/login/login.md", "/path/manual/user/firstUser")
        ('login', 'login', 'login')
    """
    # 절대 경로로 변환
    abs_file_path = os.path.abspath(file_path)
    abs_root_path = os.path.abspath(root_path)
    
    # 상대 경로 계산
    rel_path = os.path.relpath(abs_file_path, abs_root_path)
    
    # 파일명과 확장자 분리
    file_name_with_ext = os.path.basename(abs_file_path)
    file_name = os.path.splitext(file_name_with_ext)[0]
    
    # 디렉토리명 추출
    dir_name = os.path.basename(os.path.dirname(abs_file_path))
    
    return (rel_path, file_name, dir_name)

### 1단계 단위 테스트 코드

* 외부에 직접 노출되지 않음
1. scan_directory       <- process_markdown_files
2. read_markdown_file   <- process_markdown_files
3. extract_path_info    <- process_markdown_files

In [31]:

# 1단계 통합 테스트 - 셀 단위 실행으로 변수 공유
print("1단계 함수 테스트 시작")

# 테스트 경로 설정 (전역 변수로 사용)
test_path = "./manual/user/firstUser"
print(f"테스트 경로: {test_path}")
print("-" * 50)

1단계 함수 테스트 시작
테스트 경로: ./manual/user/firstUser
--------------------------------------------------


In [32]:
# 1-1. scan_directory 테스트
print("1. scan_directory() 테스트")
md_files = scan_directory(test_path)
print(f"발견된 파일 수: {len(md_files)}")

if md_files:
    print("발견된 모든 파일들:")
    for i, file_path in enumerate(md_files, 1):
        print(f"  {i}. {os.path.basename(file_path)} ({os.path.dirname(file_path)})")
else:
    print("발견된 파일이 없습니다.")

print()
print("✅ 1-1단계 완료: md_files 변수 생성됨")


1. scan_directory() 테스트
발견된 파일 수: 4
발견된 모든 파일들:
  1. approval.md (/Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/approval)
  2. login.md (/Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/login)
  3. namespaces.md (/Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/namespaces)
  4. project.md (/Users/gu.han/Documents/AI.WORK/RAG_Master/manual/user/firstUser/project)

✅ 1-1단계 완료: md_files 변수 생성됨


In [33]:
# 1-2. read_markdown_file 테스트 (md_files 변수 활용)
print("2. read_markdown_file() 테스트")
test_file_contents = {}  # 파일별 내용 저장

if md_files:
    # 첫 번째 파일로 상세 테스트
    test_file = md_files[0]
    print(f"테스트 파일: {os.path.basename(test_file)}")
    
    content = read_markdown_file(test_file)
    if content:
        test_file_contents[test_file] = content
        lines = content.split('\n')
        print(f"파일 내용 길이: {len(content)}자")
        print(f"총 라인 수: {len(lines)}")
        print("첫 5줄 미리보기:")
        for i, line in enumerate(lines[:5], 1):
            print(f"  {i}: {line}")
    else:
        print("파일 읽기 실패")
    
    # 모든 파일 읽기 테스트
    print(f"\n모든 파일 읽기 테스트:")
    for file_path in md_files:
        content = read_markdown_file(file_path)
        if content:
            test_file_contents[file_path] = content
            print(f"  ✅ {os.path.basename(file_path)}: {len(content)}자")
        else:
            print(f"  ❌ {os.path.basename(file_path)}: 읽기 실패")
else:
    print("테스트할 파일이 없습니다.")

print()
print(f"✅ 1-2단계 완료: test_file_contents 변수 생성됨 ({len(test_file_contents)}개 파일)")


2. read_markdown_file() 테스트
테스트 파일: approval.md
파일 내용 길이: 1402자
총 라인 수: 85
첫 5줄 미리보기:
  1: # 신청관리
  2: 
  3: > 신청관리는 신청한 네임스페이스 목록 확인하고, 결재하는 서비스 입니다.
  4: > 
  5: > 운영자 권한을 가진 사용자는 모른 목록이 표시됩니다. <br/>

모든 파일 읽기 테스트:
  ✅ approval.md: 1402자
  ✅ login.md: 709자
  ✅ namespaces.md: 1966자
  ✅ project.md: 755자

✅ 1-2단계 완료: test_file_contents 변수 생성됨 (4개 파일)


In [34]:
# 1-3. extract_path_info 테스트 (md_files 변수 활용)
print("3. extract_path_info() 테스트")
test_path_info = {}  # 파일별 경로 정보 저장

if md_files:
    print("모든 파일의 경로 정보:")
    for i, file_path in enumerate(md_files, 1):
        rel_path, file_name, dir_name = extract_path_info(file_path, test_path)
        
        # 경로 정보 저장
        test_path_info[file_path] = {
            'rel_path': rel_path,
            'file_name': file_name,
            'dir_name': dir_name
        }
        
        print(f"  파일 {i}: {file_name}.md")
        print(f"    상대경로: {rel_path}")
        print(f"    파일명: {file_name}")
        print(f"    디렉토리명: {dir_name}")
        print()
else:
    print("테스트할 파일이 없습니다.")

print("-" * 50)
print("1단계 함수 테스트 완료")
print(f"✅ 1-3단계 완료: test_path_info 변수 생성됨 ({len(test_path_info)}개 파일)")
print()
print("📋 1단계 결과 변수:")
print(f"  - test_path: {test_path}")
print(f"  - md_files: {len(md_files) if md_files else 0}개 파일")
print(f"  - test_file_contents: {len(test_file_contents)}개 파일 내용")
print(f"  - test_path_info: {len(test_path_info)}개 파일 경로 정보")


3. extract_path_info() 테스트
모든 파일의 경로 정보:
  파일 1: approval.md
    상대경로: approval/approval.md
    파일명: approval
    디렉토리명: approval

  파일 2: login.md
    상대경로: login/login.md
    파일명: login
    디렉토리명: login

  파일 3: namespaces.md
    상대경로: namespaces/namespaces.md
    파일명: namespaces
    디렉토리명: namespaces

  파일 4: project.md
    상대경로: project/project.md
    파일명: project
    디렉토리명: project

--------------------------------------------------
1단계 함수 테스트 완료
✅ 1-3단계 완료: test_path_info 변수 생성됨 (4개 파일)

📋 1단계 결과 변수:
  - test_path: ./manual/user/firstUser
  - md_files: 4개 파일
  - test_file_contents: 4개 파일 내용
  - test_path_info: 4개 파일 경로 정보


### 2단계: 파싱 모듈

In [35]:
import re
from typing import List, Tuple, Optional

# =============================================================================
# 2단계: 파싱 모듈 (핵심 2개 함수만)
# =============================================================================

def parse_markdown_lines(content: str) -> List[str]:
    """
    마크다운 내용을 라인별로 분할합니다.
    
    Args:
        content (str): 마크다운 파일 내용
    
    Returns:
        List[str]: 라인별로 분할된 내용
    
    Example:
        >>> content = "# Title\n\n## Section\nContent here"
        >>> parse_markdown_lines(content)
        ['# Title', '', '## Section', 'Content here']
    """
    if not content:
        return []
    
    # 라인별로 분할하고 오른쪽 공백 제거
    lines = [line.rstrip() for line in content.split('\n')]
    return lines

In [36]:
def detect_header_level(line: str) -> Tuple[Optional[int], Optional[str]]:
    """
    라인에서 마크다운 헤더 레벨과 제목을 감지합니다.
    
    Args:
        line (str): 검사할 라인
    
    Returns:
        Tuple[Optional[int], Optional[str]]: (헤더레벨, 제목) 또는 (None, None)
    """
    # 마크다운 헤더 패턴 매칭 (# ~ ######)
    header_pattern = r'^(#{1,6})\s+(.+)$'
    match = re.match(header_pattern, line.strip())
    
    if match:
        header_level = len(match.group(1))  # # 개수
        header_title = match.group(2).strip()  # 제목 부분
        return (header_level, header_title)
    
    return (None, None)


In [38]:
from typing import List, Dict, Optional, Any


def process_markdown_files(root_path: str) -> List[Dict[str, any]]:
    """
    1단계 함수들을 사용해서 마크다운 파일들을 파싱합니다.
    
    Args:
        root_path (str): 스캔할 루트 디렉토리 경로
    
    Returns:
        List[Dict]: 파싱된 파일 정보 리스트
        각 딕셔너리는 다음 키를 포함:
        - 'file_path': 파일 경로
        - 'rel_path': 상대 경로  
        - 'file_name': 파일명
        - 'dir_name': 디렉토리명
        - 'lines': 파싱된 라인 리스트
        - 'headers': 감지된 헤더 정보 리스트
    """
    # 1단계 함수 사용: 파일 스캔
    md_files = scan_directory(root_path)
    
    parsed_files = []
    
    for file_path in md_files:
        print(f"📖 파싱 중: {file_path}")
        
        # 1단계 함수 사용: 파일 읽기
        content = read_markdown_file(file_path)
        if content is None:
            continue
            
        # 1단계 함수 사용: 경로 정보 추출
        rel_path, file_name, dir_name = extract_path_info(file_path, root_path)
        
        # 2단계 함수 사용: 라인별 파싱
        lines = parse_markdown_lines(content)
        
        # 2단계 함수 사용: 헤더 감지
        headers = []
        for line_num, line in enumerate(lines):
            level, title = detect_header_level(line)
            if level is not None:
                headers.append({
                    'line_num': line_num,
                    'level': level,
                    'title': title,
                    'line': line
                })
        
        # 파싱 결과 저장
        parsed_file = {
            'file_path': file_path,
            'rel_path': rel_path,
            'file_name': file_name,
            'dir_name': dir_name,
            'lines': lines,
            'headers': headers,
            'total_lines': len(lines),
            'header_count': len(headers)
        }
        
        parsed_files.append(parsed_file)
    
    return parsed_files

### 2단계 단위&통합 테스트 코드

(public)Parse_markdown_lines() → (private)detect_header_level() → (private)process_markdown_files()

In [40]:
# 2-1. parse_markdown_lines 통합테스트 (1단계 함수 활용)
print("2단계 함수 테스트 시작 (1단계 결과 활용)")
print("-" * 50)

# 1단계 결과 확인
print("📋 1단계 결과 확인:")
print(f"  - 사용 가능한 파일: {len(test_file_contents)}개")
if test_file_contents:
    first_file = list(test_file_contents.keys())[0]
    print(f"  - 첫 번째 파일: {os.path.basename(first_file)}")
print()

# 실제 파일 내용으로 parse_markdown_lines 테스트
print("1. parse_markdown_lines() 테스트 (실제 파일 사용)")
test_parsed_lines = {}  # 파일별 파싱된 라인 저장

if test_file_contents:
    for file_path, content in test_file_contents.items():
        lines = parse_markdown_lines(content)
        test_parsed_lines[file_path] = lines
        
        file_name = os.path.basename(file_path)
        print(f"  📄 {file_name}:")
        print(f"    - 원본 길이: {len(content)}자")
        print(f"    - 파싱된 라인 수: {len(lines)}")
        print(f"    - 첫 3줄: {[line[:30] + '...' if len(line) > 30 else line for line in lines[:3]]}")
        print()

print(f"✅ 2-1단계 완료: test_parsed_lines 변수 생성됨 ({len(test_parsed_lines)}개 파일)")

2단계 함수 테스트 시작 (1단계 결과 활용)
--------------------------------------------------
📋 1단계 결과 확인:
  - 사용 가능한 파일: 4개
  - 첫 번째 파일: approval.md

1. parse_markdown_lines() 테스트 (실제 파일 사용)
  📄 approval.md:
    - 원본 길이: 1402자
    - 파싱된 라인 수: 85
    - 첫 3줄: ['# 신청관리', '', '> 신청관리는 신청한 네임스페이스 목록 확인하고, 결재...']

  📄 login.md:
    - 원본 길이: 709자
    - 파싱된 라인 수: 49
    - 첫 3줄: ['# Login', '', '> PaaS Portal을 사용하기 위해 계정 인증을 ...']

  📄 namespaces.md:
    - 원본 길이: 1966자
    - 파싱된 라인 수: 58
    - 첫 3줄: ['# Namespaces', '', '> Namespaces는 상단의 클러스터에서 서비스중인...']

  📄 project.md:
    - 원본 길이: 755자
    - 파싱된 라인 수: 47
    - 첫 3줄: ['# Project', '', '> 사용자에게 할당된 Project와 하위의 Names...']

✅ 2-1단계 완료: test_parsed_lines 변수 생성됨 (4개 파일)


In [42]:
# 2-2. detect_header_level 단위 테스트 (test_parsed_lines 활용)
print("2. detect_header_level() 테스트 (파싱된 라인 사용)")
test_headers = {}  # 파일별 헤더 정보 저장

if test_parsed_lines:
    for file_path, lines in test_parsed_lines.items():
        file_headers = []
        
        for line_num, line in enumerate(lines):
            level, title = detect_header_level(line)
            if level is not None:
                file_headers.append({
                    'line_num': line_num,
                    'level': level,
                    'title': title,
                    'line': line
                })
        
        test_headers[file_path] = file_headers
        
        file_name = os.path.basename(file_path)
        print(f"  📄 {file_name}:")
        print(f"    - 총 라인 수: {len(lines)}")
        print(f"    - 검출된 헤더 수: {len(file_headers)}")
        
        if file_headers:
            print("    - 헤더 목록:")
            for header in file_headers:
                indent = "    " + "  " * header['level']
                print(f"{indent}└ 레벨 {header['level']}: {header['title']} (라인 {header['line_num']+1})")
        else:
            print("    - 헤더 없음")
        print()

print(f"✅ 2-2단계 완료: test_headers 변수 생성됨 ({len(test_headers)}개 파일)")

# 헤더 검출 통계
total_headers = sum(len(headers) for headers in test_headers.values())
print(f"📊 헤더 검출 통계: 총 {total_headers}개 헤더 검출됨")


2. detect_header_level() 테스트 (파싱된 라인 사용)
  📄 approval.md:
    - 총 라인 수: 85
    - 검출된 헤더 수: 6
    - 헤더 목록:
      └ 레벨 1: 신청관리 (라인 1)
        └ 레벨 2: **목차** (라인 12)
        └ 레벨 2: 신청 조회 (라인 18)
          └ 레벨 3: 신청 목록 (라인 21)
          └ 레벨 3: 신청 상세정보 (라인 52)
        └ 레벨 2: 신청 결재 (라인 67)

  📄 login.md:
    - 총 라인 수: 49
    - 검출된 헤더 수: 4
    - 헤더 목록:
      └ 레벨 1: Login (라인 1)
        └ 레벨 2: 목차 (라인 10)
        └ 레벨 2: 1. 로그인 페이지 (라인 17)
        └ 레벨 2: 2. 로그인을 통한 인증 (라인 27)

  📄 namespaces.md:
    - 총 라인 수: 58
    - 검출된 헤더 수: 5
    - 헤더 목록:
      └ 레벨 1: Namespaces (라인 1)
        └ 레벨 2: **목차** (라인 6)
        └ 레벨 2: 1. Namespace 메뉴 진입 (라인 11)
        └ 레벨 2: 2. Namespace 생성 신청 (라인 22)
        └ 레벨 2: 3. Namespace 생성 확인 (라인 50)

  📄 project.md:
    - 총 라인 수: 47
    - 검출된 헤더 수: 5
    - 헤더 목록:
      └ 레벨 1: Project (라인 1)
        └ 레벨 2: 목차 (라인 8)
        └ 레벨 2: 1. Project 메뉴 확인 (라인 16)
        └ 레벨 2: 2. Project 생성 (라인 29)
        └ 레벨 2: 3. Project 생성 확인 (라인 41)

✅ 2-2단계 완료: test_head

In [ ]:
# 2-3. process_markdown_files 통합 테스트 (1단계 결과와 비교 검증)
print("3. process_markdown_files() 통합 테스트 (1단계 결과 비교)")
print("-" * 30)

# process_markdown_files 함수 실행 (1단계 함수들을 내부적으로 호출)
parsed_files_result = process_markdown_files(test_path)

print(f"처리된 파일 수: {len(parsed_files_result)}")
print()

# 1단계 결과와 2단계 결과 비교 검증
print("📊 1단계 vs 2단계 결과 비교:")
print(f"  - 1단계 파일 수: {len(md_files)}")
print(f"  - 2단계 파일 수: {len(parsed_files_result)}")
print(f"  - 파일 수 일치: {'✅' if len(md_files) == len(parsed_files_result) else '❌'}")
print()

# 각 파일별 상세 비교
for i, parsed_file in enumerate(parsed_files_result):
    file_path = parsed_file['file_path']
    file_name = parsed_file['file_name']
    
    print(f"파일 {i+1}: {file_name}.md")
    print(f"  📍 경로: {parsed_file['rel_path']}")
    print(f"  📊 총 라인 수: {parsed_file['total_lines']}")
    print(f"  🏷️ 헤더 수: {parsed_file['header_count']}")
    
    # 1단계 결과와 비교
    if file_path in test_file_contents:
        original_content = test_file_contents[file_path]
        original_lines = len(original_content.split('\n'))
        print(f"  🔍 1단계 라인 수: {original_lines}")
        print(f"  ✅ 라인 수 일치: {'✅' if original_lines == parsed_file['total_lines'] else '❌'}")
        
        # 헤더 정보와 비교
        if file_path in test_headers:
            manual_headers = test_headers[file_path]
            print(f"  🔍 수동 헤더 검출: {len(manual_headers)}개")
            print(f"  ✅ 헤더 수 일치: {'✅' if len(manual_headers) == parsed_file['header_count'] else '❌'}")
    
    # 헤더 레벨별 분포
    if parsed_file['headers']:
        level_count = {}
        for header in parsed_file['headers']:
            level = header['level']
            level_count[level] = level_count.get(level, 0) + 1
        
        print("  📈 헤더 레벨 분포:")
        for level in sorted(level_count.keys()):
            print(f"    레벨 {level}: {level_count[level]}개")
    
    print()

print("-" * 50)
print("2단계 통합 테스트 완료")
print()
print("📋 2단계 최종 결과 변수:")
print(f"  - test_parsed_lines: {len(test_parsed_lines)}개 파일의 파싱된 라인")
print(f"  - test_headers: {len(test_headers)}개 파일의 헤더 정보")
print(f"  - parsed_files_result: {len(parsed_files_result)}개 파일의 통합 결과")

# 3단계로 전달할 변수 준비
step2_output = parsed_files_result
print(f"  - step2_output: 3단계로 전달할 데이터 준비 완료")


### 3단계: 섹션 처리 모듈

In [68]:
from typing import List, Dict, Optional, Any

# =============================================================================
# 3단계: 섹션 처리 모듈 (2단계 출력을 입력으로 받음)
# =============================================================================

def should_skip_section(section_title: str) -> bool:
    """
    섹션을 스킵해야 하는지 판단합니다.
    
    Args:
        section_title (str): detect_header_level()에서 받은 섹션 제목
    
    Returns:
        bool: 스킵해야 하면 True, 처리해야 하면 False
    
    Example:
        >>> should_skip_section("목차")
        True
        >>> should_skip_section("1. 로그인 페이지")
        False
    """
    skip_patterns = [
        "목차",
        "**목차**",
        "table of contents",
        "toc",
        "contents"
    ]
    
    # 대소문자 구분 없이 비교
    section_lower = section_title.lower().strip()
    
    for pattern in skip_patterns:
        if pattern.lower() in section_lower:
            return True
    
    return False

def create_section(header: Dict[str, Any], content: List[str]) -> Dict[str, Any]:
    """
    헤더 정보와 내용을 받아서 섹션 객체를 생성합니다.
    
    Args:
        header (Dict): detect_header_level()에서 받은 헤더 정보
                      {'line_num': int, 'level': int, 'title': str}
        content (List[str]): parse_markdown_lines()에서 받은 해당 섹션의 내용 라인들
    
    Returns:
        Dict[str, Any]: 섹션 정보
        {
            'level': int,           # 헤더 레벨
            'title': str,           # 섹션 제목
            'content': List[str],   # 섹션 내용 (빈 라인 제거됨)
            'line_start': int,      # 시작 라인 번호
            'line_end': int,        # 종료 라인 번호
            'content_length': int,  # 내용 길이
            'is_empty': bool        # 빈 섹션 여부
        }
    
    Example:
        >>> header = {'line_num': 5, 'level': 2, 'title': '1. 로그인 페이지'}
        >>> content = ['브라우저에서 접속합니다.', '', '로그인 화면이 나타납니다.']
        >>> create_section(header, content)
        {'level': 2, 'title': '1. 로그인 페이지', 'content': [...], ...}
    """
    # 빈 라인 제거 및 내용 정리
    cleaned_content = []
    for line in content:
        line_stripped = line.strip()
        if line_stripped:  # 빈 라인이 아닌 경우만 추가
            cleaned_content.append(line)
    
    # 섹션 정보 생성
    section = {
        'level': header['level'],
        'title': header['title'],
        'content': cleaned_content,
        'line_start': header['line_num'],
        'line_end': header['line_num'] + len(content) - 1,
        'content_length': len(cleaned_content),
        'is_empty': len(cleaned_content) == 0
    }
    
    return section

def extract_sections_from_parsed_data(lines: List[str], headers: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    2단계에서 받은 파싱 데이터로부터 섹션들을 추출합니다.
    
    Args:
        lines (List[str]): parse_markdown_lines()에서 받은 라인 리스트
        headers (List[Dict]): detect_header_level()에서 받은 헤더 리스트
    
    Returns:
        List[Dict[str, Any]]: 처리된 섹션 리스트 (스킵된 섹션 제외, 레벨 2만 포함)
    """
    sections = []
    
    # 레벨 2 헤더만 필터링
    level2_headers = [h for h in headers if h['level'] == 2]
    
    for i, header in enumerate(level2_headers):
        # 스킵 대상 섹션 확인
        if should_skip_section(header['title']):
            print(f"   ⏭️ 스킵: {header['title']}")
            continue
        
        # 현재 섹션의 시작과 끝 라인 계산
        start_line = header['line_num'] + 1  # 헤더 다음 라인부터
        
        if i + 1 < len(level2_headers):
            # 다음 헤더가 있는 경우: 다음 헤더 직전까지
            end_line = level2_headers[i + 1]['line_num']
        else:
            # 마지막 헤더인 경우: 파일 끝까지
            end_line = len(lines)
        
        # 섹션 내용 추출
        section_content = lines[start_line:end_line]
        
        # 섹션 생성
        section = create_section(header, section_content)
        
        # 빈 섹션이 아닌 경우만 추가
        if not section['is_empty']:
            sections.append(section)
            print(f"   ✅ 생성: {section['title']} ({section['content_length']} 라인)")
        else:
            print(f"   ⚠️ 빈 섹션 제외: {section['title']}")
    
    return sections

In [ ]:
test_path = "./manual/user"
